In [1]:
import pandas as pd
import os
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.metrics import mean_squared_error
import warnings
import pickle

In [2]:
def read_parquet_file(filename):
    """
    To read the data frame from parquet file
    
    """
    
    df = pd.read_parquet(filename)
    
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df.duration.apply(lambda td : td.total_seconds()/60)
    
    df = df[((df.duration >= 1) & (df.duration <=60))]
    
    categorical = ['PULocationID','DOLocationID']
        
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_parquet_file('/home/bcthakredata/mlops_zoomcamp/week1/data/green_tripdata_2022-01.parquet')
df_val = read_parquet_file('/home/bcthakredata/mlops_zoomcamp/week1/data/green_tripdata_2022-02.parquet')

In [5]:
df_train['PU_DO_PAIR'] = df_train['PULocationID'] + "_" + df_train['DOLocationID']
df_val['PU_DO_PAIR'] = df_val['PULocationID'] + "_" + df_val['DOLocationID']

In [6]:
categorical = ['PULocationID','DOLocationID','PU_DO_PAIR']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
with mlflow.start_run():

    mlflow.set_tag("developer", "bt")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")